In [1]:
import os

file_path = "./data/gpt-4-turbo-preview/gpt-4/seed0/summary.csv"

API_KEY = os.environ["OPENAI_API_KEY"]

In [2]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.colors import to_rgba, to_hex
import imageio
import os

# Load the CSV file
file_path = "./data/gpt-4-turbo-preview/gpt-4/seed0/summary.csv"
game_summary = pd.read_csv(file_path)

# Create directory for saving images
output_dir = "./graph_images/temp"
os.makedirs(output_dir, exist_ok=True)

# Extract the current room from obs_text
game_summary["current_room"] = game_summary["obs_text"].str.extract(
    r"You are currently in Room (\d+)"
)

# Remove rows where current_room is NaN
game_summary = game_summary.dropna(subset=["current_room"])


# Parse bomb information from obs_text
def parse_bomb_info(text):
    if pd.isna(text):
        return None
    if "Results: You inspected Bomb" in text:
        bomb_info_part = text.split("Results: You inspected Bomb")[-1]
        sequence_part = bomb_info_part.split("remaining sequence is ")[-1]
        colors = sequence_part.split(".")[0].split("-")
        return [color.lower() for color in colors]
    return None


game_summary["bomb_colors"] = game_summary["obs_text"].apply(parse_bomb_info)


# Function to blend colors
def blend_colors(colors):
    if not colors:
        return "grey"  # Default color for fully defused bombs or no color
    rgba_colors = [to_rgba(color.lower()) for color in colors]
    blended_rgba = [sum(channel) / len(channel) for channel in zip(*rgba_colors)]
    return to_hex(blended_rgba)


# Initialize bomb states
bomb_states = {}

# Initialize the graph with all nodes and edges
G = nx.Graph()

# Group by agent and deduce their paths
agent_paths = game_summary.groupby("agent_id")["current_room"].apply(list).reset_index()

# Create a list of edges based on the paths, excluding self-loops
edges = []
for path in agent_paths["current_room"]:
    for i in range(len(path) - 1):
        if path[i] != path[i + 1]:
            edges.append((path[i], path[i + 1]))

G.add_edges_from(edges)

# Initialize bomb states
initial_bomb_info = game_summary.dropna(subset=["bomb_colors"]).drop_duplicates(
    subset=["current_room"]
)
for index, row in initial_bomb_info.iterrows():
    bomb_states[row["current_room"]] = row["bomb_colors"]

# Compute fixed layout
pos = nx.spring_layout(G)


# Draw initial graph without displaying it
def draw_initial_graph():
    plt.figure(figsize=(16, 8))
    # Draw nodes and edges
    nx.draw(
        G, pos, with_labels=True, node_color="skyblue", font_size=15, font_weight="bold"
    )
    # Draw initial bomb colors
    for room, colors in bomb_states.items():
        blended_color = blend_colors(colors)
        nx.draw_networkx_nodes(
            G, pos, nodelist=[room], node_color=blended_color, node_size=500
        )
    plt.title("Initial Rooms and Bomb Locations")
    plt.close()


draw_initial_graph()


# Function to update the graph for each round
def update_graph(round_number, actions_text, agent_positions):
    plt.figure(figsize=(16, 8))
    # Draw nodes and edges
    nx.draw(
        G, pos, with_labels=True, node_color="skyblue", font_size=15, font_weight="bold"
    )
    # Update bomb colors
    for room, colors in bomb_states.items():
        blended_color = blend_colors(colors)
        nx.draw_networkx_nodes(
            G, pos, nodelist=[room], node_color=blended_color, node_size=500
        )

    # Draw agent positions with distinct offsets to avoid overlapping
    offset = 0.03
    for i, (agent, position) in enumerate(agent_positions.items()):
        agent_pos = pos[position]
        vertical_offset = (i % 3) * offset
        vertical_offset -= 0.05
        horizontal_offset = offset
        nx.draw_networkx_nodes(
            G, pos, nodelist=[position], node_color="red", node_size=600, alpha=0.0
        )
        plt.text(
            agent_pos[0] + horizontal_offset,
            agent_pos[1] + vertical_offset,
            agent,
            fontsize=12,
            color="black",
            fontweight="bold",
        )

    plt.text(
        0.05,
        0.95,
        f"Round {round_number}",
        transform=plt.gca().transAxes,
        fontsize=20,
        verticalalignment="top",
    )
    plt.title(f"Rooms, Their Connectivity, and Bomb Locations - Round {round_number}")

    # Add actions text on the side
    plt.figtext(
        0.8,
        0.5,
        actions_text,
        wrap=True,
        horizontalalignment="left",
        fontsize=14,
        bbox=dict(facecolor="white", alpha=0.5),
    )

    plt.savefig(os.path.join(output_dir, f"round_{round_number}.png"))
    plt.close()


# Process each round and update the graph
for round_number in sorted(game_summary["round"].unique()):
    round_data = game_summary[game_summary["round"] == round_number]
    actions_text = ""
    agent_positions = {}
    # Update bomb states based on actions
    for index, row in round_data.iterrows():
        if row["bomb_colors"] is not None:
            bomb_states[row["current_room"]] = row["bomb_colors"]
        elif "use" in row["action"] and "tool" in row["action"]:
            room = row["current_room"]
            tool_color = row["action"].split(" ")[-2].lower()
            if room in bomb_states and bomb_states[room] is not None:
                if tool_color in bomb_states[room]:
                    bomb_states[room].remove(tool_color)
                    if not bomb_states[room]:  # If all colors are removed
                        bomb_states[room] = None  # Mark as fully defused
        # Collect actions for the text
        actions_text += (
            f"Agent {row['agent_id']} in Room {row['current_room']}: {row['action']}\n"
        )
        agent_positions[row["agent_id"]] = row["current_room"]

    # Add edges and nodes for the current round
    for path in round_data.groupby("agent_id")["current_room"].apply(list):
        for i in range(len(path) - 1):
            if path[i] != path[i + 1]:
                G.add_edge(path[i], path[i + 1])
    # Ensure all nodes have positions
    for room in bomb_states.keys():
        G.add_node(room)
    # Update the graph for the current round
    update_graph(round_number, actions_text, agent_positions)

# Create GIF
images = []
for round_number in sorted(game_summary["round"].unique()):
    images.append(
        imageio.v2.imread(os.path.join(output_dir, f"round_{round_number}.png"))
    )
imageio.mimsave(
    "./graph_images/rooms_and_bombs.gif", images, duration=4000
)  # Slower iteration

# Remove temporary images
for round_number in sorted(game_summary["round"].unique()):
    os.remove(os.path.join(output_dir, f"round_{round_number}.png"))

In [3]:
test_formatter = ["str1", "str2"]